In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score 

# from zipfile import ZipFile
# import os,glob
# from skimage.io import imread
# from skimage.transform import resize 
# import matplotlib.pyplot as plt
# import random
# import warnings
# from scipy import ndarray
# import skimage as sk
# from skimage import transform
# from skimage import util
# from skimage import io
# from sklearn import metrics
# from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import keras
# from keras import layers
# from keras.models import Model
# from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split
# from imgaug import augmenters as iaa
# import random

# from keras.models import Sequential
# from keras.layers import Dense, Activation

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(rescale=1./255)
train_generator=data_generator.flow_from_directory(
	'/content/drive/MyDrive/생체인증보안/01_finger_training',
	target_size=(128,128),
	batch_size=80,
	class_mode='categorical')

x_data, y_data = train_generator.next()
print(x_data.shape, y_data.shape)

In [ ]:
# augs = [x_data[79]] * 9

# seq1 = iaa.Sequential([
#     iaa.Affine(
#         scale={"x": (1.5, 1.5), "y": (1.5, 1.5)},
#     )
# ], random_order=True)

# seq2 = iaa.Sequential([
#     # blur images with a sigma of 0 to 0.5
#     iaa.GaussianBlur(sigma=(0, 0.5)),
#     iaa.Affine(
#         # scale images to 90-110% of their size, individually per axis
#         scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
#         # translate by -10 to +10 percent (per axis)
#         translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
#         # rotate by -30 to +30 degrees
#         rotate=(-30, 30),
#         # use nearest neighbour or bilinear interpolation (fast)
#         order=[0, 1],
#         # if mode is constant, use a cval between 0 and 255
#         cval=255
#     )
# ], random_order=True)

# x1_batch = seq1.augment_images(x_test)
# x1_batch = seq2.augment_images(x_test)

# augs = seq1.augment_images(augs)
# augs = seq2.augment_images(augs)

# plt.figure(figsize=(16, 6))
# plt.subplot(2, 5, 1)
# plt.title('original')
# plt.imshow(x_data[0].squeeze(), cmap='gray')
# for i, aug in enumerate(augs):
#     plt.subplot(2, 5, i+2)
#     plt.title('aug %02d' % int(i+1))
#     plt.imshow(aug.squeeze(), cmap='gray')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,shuffle=True,random_state=1000)

print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)


In [ ]:
m1=Sequential()
m1.add(BatchNormalization(input_shape = (128,128,3)))
m1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
m1.add(MaxPooling2D(pool_size=2))
m1.add(Convolution2D(filters=6,kernel_size=4,padding='same',activation='relu'))
m1.add(MaxPooling2D(pool_size=2))
m1.add(Convolution2D(filters=128,kernel_size=3,padding='same',activation='relu'))
m1.add(MaxPooling2D(pool_size=2))
m1.add(Convolution2D(filters=128,kernel_size=2,padding='same',activation='relu'))
m1.add(MaxPooling2D(pool_size=2))
m1.add(Flatten()) 
m1.add(Dense(units=128,activation = 'relu'))
m1.add(Dense(units = 64, activation = 'relu'))
m1.add(Dense(units = 32, activation = 'relu'))
m1.add(Dense(units = 8, activation = 'softmax'))

m1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

model = m1.fit(x_train,y_train,epochs=60,validation_data=(x_test,y_test),verbose = 1, initial_epoch=0)

In [ ]:
loss_and_metrics = m1.evaluate(x_test,y_test)

print(loss_and_metrics)

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255)
train_generator=data_generator.flow_from_directory(
	'/content/drive/MyDrive/생체인증보안/01_finger_test',
	target_size=(128,128),
	batch_size=80,
	class_mode='categorical')
x_testset, y_testset = train_generator.next()

# print(x_testset[0])

predicted = m1.predict(x_testset, batch_size=80)
predicted = np.argmax(np.round(predicted),axis=1)
print("Prediction :",predicted)
